In [24]:
from bs4 import BeautifulSoup
import requests
# from tabulate import tabulate
from multiprocessing import Process

print("Fetching.....", end="")


def GetProxys():
    website_html = requests.get("https://free-proxy-list.net").text
    soup = BeautifulSoup(website_html, "html.parser")
    soup = soup.find("table")
    Headings = []
    Body = []
    Ip = []
    for heading in soup.find_all("th"):
        Headings.append(heading.text)
    for ip_html in soup.find_all("tr"):
        ip_info_list = []
        for ip_info in ip_html.find_all("td"):
            ip_info_list.append(ip_info.text)
        if len(ip_info_list) > 1:
            if ip_info_list[6] == "yes":
                Ip.append(f"{ip_info_list[0]}:{ip_info_list[1]}")
                Body.append(ip_info_list)
    return Ip, Body


Ips, info = GetProxys()
print("[Done]")



Fetching.....[Done]


In [25]:
def perform_request(ip):
    http_proxy = f"http://{ip}"
    https_proxy = f"https://{ip}"

    proxyDict = {
        "http": http_proxy,
        "https": https_proxy,
    }
    for _ in range(2):
        try:
            r = requests.get("http://api.ipify.org",
                             proxies=proxyDict, timeout=10)
            print(f"{r}/{r.text}", end="/")
        except:
            print("{-_-}", end="/")
    print()

In [26]:
def divide_chunks(l, n):
    list = []
    for x in range(0, len(l), n):
        list.append(l[x:x+5])
    return list

Chunks = divide_chunks(Ips, 6)

def Check_availability(list):
    for x in list:
        perform_request(x)

for Ip_list in Chunks:
    p1=Process(target=Check_availability,args=(Ip_list,))
    p1.start()
    p1.join()